In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import cv2
import os
import warnings
warnings.filterwarnings('ignore')
from glob import glob


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
IMAGE_SIZE = [224,224]
BS = 32

In [11]:
train='../input/cotton-disease-dataset/Cotton Disease/train'
test='../input/cotton-disease-dataset/Cotton Disease/test'
val='../input/cotton-disease-dataset/Cotton Disease/val'

In [12]:
mob = MobileNetV2(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

In [13]:
for layer in mob.layers:
    layer.trainable = False

In [14]:
folders = glob('../input/cotton-disease-dataset/Cotton Disease/train/*')

In [17]:
x = Flatten()(mob.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=mob.input, outputs=prediction)

In [18]:
model.summary()

In [19]:
from tensorflow.keras.utils import plot_model

In [20]:
plot_model(model,to_file="Cotton Disease Model.png",show_shapes=True,show_dtype=True,dpi=300)

In [37]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=Adam(lr=0.00002),
  metrics=['accuracy']
)

In [38]:
datagen = ImageDataGenerator(zoom_range= 0.2,
                             horizontal_flip= True,                             
                             rescale= 1./255,
                             shear_range = 0.2)

In [39]:
train = datagen.flow_from_directory('../input/cotton-disease-dataset/Cotton Disease/train',
                                   color_mode='rgb',
                                   class_mode='categorical',
                                   batch_size=BS,
                                   target_size=(224,224))

In [40]:
valid = datagen.flow_from_directory('../input/cotton-disease-dataset/Cotton Disease/val',
                                   color_mode='rgb',
                                   class_mode='categorical',
                                   batch_size=BS,
                                   target_size=(224,224))

In [41]:
datagen1 = ImageDataGenerator(rescale= 1./255)

test = datagen1.flow_from_directory('../input/cotton-disease-dataset/Cotton Disease/test',
                                   color_mode='rgb',
                                   class_mode='categorical',
                                   batch_size=BS,
                                   target_size=(224,224))

In [42]:
classes = list(train.class_indices.keys())
classes

In [43]:
for img,labels in train:
    r=labels.flatten()
    break

In [44]:
r

In [32]:
labels = classes
img_size = 224
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [33]:
train = get_data('../input/cotton-disease-dataset/Cotton Disease/train')
val = get_data('../input/cotton-disease-dataset/Cotton Disease/val')

In [34]:
# 'diseased cotton leaf': 0,
#  'diseased cotton plant': 1,
#  'fresh cotton leaf': 2,
#  'fresh cotton plant': 3

l = []
for i in train:
    if(i[1] == 0):
        l.append("diseased cotton leaf")
    elif i[1] == 1:
        l.append("diseased cotton plant")
    elif i[1] == 2:
        l.append("fresh cotton leaf")
    elif i[1] == 3:
        l.append("fresh cotton plant")

In [35]:
plt.figure(figsize=(10,7))
sns.set_style('darkgrid')
sns.countplot(l)

In [45]:
history = model.fit(train,batch_size=BS,epochs=10,validation_data=valid,validation_steps=len(valid),steps_per_epoch=len(train))

In [46]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [47]:
model.evaluate(test)

In [ ]:
model.save('./cottonmobilenet.h5')

In [ ]:
clear_session()